In [67]:
import numpy as np
import pandas as pd
import pyvo

In [91]:
service = pyvo.dal.TAPService('https://irsa.ipac.caltech.edu/TAP')
mass_2_sample = service.run_async("""
           SELECT ra,dec,j_m,j_msigcom,h_m,h_msigcom,k_m,k_msigcom,ph_qual,cc_flg
           FROM fp_psc
           WHERE CONTAINS(POINT('ICRS',ra, dec), CIRCLE('ICRS',202.48417,47.23056,0.4))=1
    """)

# Wait for 23s.

In [89]:
mass_2_sample.to_table()

ra,dec,j_m,j_msigcom,h_m,h_msigcom,k_m,k_msigcom,ph_qual,cc_flg
deg,deg,mag,mag,mag,mag,mag,mag,,
float64,float64,float32,float32,float32,float32,float32,float32,object,object
202.900750,46.961285,16.168,0.096,15.581,0.137,15.180,0.158,ABC,000
202.951614,47.024986,15.773,0.072,15.436,0.111,15.541,0.234,ABD,000
202.922589,47.024452,14.628,0.032,14.188,0.047,14.036,0.059,AAA,000
202.911833,47.011093,13.948,0.025,13.412,0.030,13.318,0.036,AAA,000
202.925932,47.004223,16.461,0.131,15.970,0.195,17.007,--,BCU,000
202.515450,46.929302,15.967,0.088,15.285,0.103,15.077,0.140,AAB,000
202.532240,46.931587,16.575,0.145,16.251,0.236,15.888,--,BDU,000
202.607930,46.932255,16.658,0.147,15.854,--,15.430,0.193,BUC,000


In [73]:
async_srv = pyvo.dal.TAPService('https://irsa.ipac.caltech.edu/TAP')
allwise_sample = async_srv.run_async("""
            SELECT  w.ra, w.dec, m.ra, m.dec
            FROM allwise_p3as_psd AS w, fp_psc AS m
            WHERE CONTAINS(POINT('ICRS', w.ra, w.dec), CIRCLE('ICRS',202.48417,47.23056,0.1))=1 AND
            CONTAINS(POINT('ICRS',m.ra, m.dec), CIRCLE('ICRS',202.48417,47.23056,0.4))=1
            """)
#job.run()

DALServiceError: Cannot wait for job completion. Job is not active!

In [72]:
allwise_sample.to_table()
#job.fetch_result()

ra,dec
deg,deg
float64,float64
202.4706382,47.3213552
202.5544249,47.1597065
202.4546241,47.1356500
202.5238645,47.3097655
202.6233414,47.2338122
202.4877581,47.3124652
202.4669864,47.3048703
202.5238501,47.3212195


In [10]:
async_srv = pyvo.dal.TAPService('https://irsa.ipac.caltech.edu/TAP')
job = async_srv.run_async("""
            SELECT TOP 10000 m.pts_key, m.ra, m.dec, m.j_m, m.j_msigcom, m.h_m, m.h_msigcom, m.k_m,
            m.k_msigcom, m.ph_qual, m.cc_flg, w.ra, w.dec, w.w1mpro, w.w2mpro,
            w.w1snr, w.w2snr
            FROM allwise_p3as_psd AS w,
            fp_psc AS m
            WHERE m.k_m - w.w2mpro >= 1.8 - 0.848 * (m.j_m - m.k_m) AND --(comment: JKW2 color cut)
            m.j_m > 0 AND
            w.w1snr >= 5 AND  --(comment: W1 and W2 S/N ratio > 5)
            w.w2snr >= 5
            """)
#job.run()

DALServiceError: Cannot wait for job completion. Job is not active!

#### Possible end product of the code

```python
async_srv = pyvo.dal.TAPService('https://irsa.ipac.caltech.edu/TAP')
job = async_srv.run_async("""
            SELECT TOP 10000 m.pts_key, m.ra, m.dec, m.j_m, m.j_msigcom, m.h_m, m.h_msigcom, m.k_m,
            m.k_msigcom, m.ph_qual, m.cc_flg, w.ra, w.dec, w.w1mpro, w.w2mpro,
            w.w1snr, w.w2snr
            FROM allwise_p3as_psd AS w
            CROSS APPLY (SELECT * FROM dbo.fGetNearestObjEq(w.ra, w.dec, 0.066)) AS x
            INNER JOIN fp_psc AS m on m.pts_key = x.pts.key
            WHERE m.k_m - w.w2mpro >= 1.8 - 0.848 * (m.j_m - m.k_m) AND --(comment: JKW2 color cut)
            m.j_m > 0 AND
            w.w1snr >= 5 AND  --(comment: W1 and W2 S/N ratio > 5)
            w.w2snr >= 5
            """)
#job.run()
#print(job.phase)
#tab = job.to_table()

In [96]:
async_srv = pyvo.dal.TAPService('https://irsa.ipac.caltech.edu/TAP')
allwise_sample = async_srv.run_async("""
            SELECT  ra, dec FROM allwise_p3as_psd
            WHERE CONTAINS(POINT('ICRS', ra, dec), CIRCLE('ICRS',202.48417,47.23056, 0.05))=1
            UNION ALL
            SELECT ra, dec FROM fp_psc
            WHERE CONTAINS(POINT('ICRS', ra, dec), CIRCLE('ICRS', 202.48417, 47.23056, 0.05))=1
            """)
#allwise_sample.run()

DALServiceError: Cannot wait for job completion. Job is not active!

In [94]:
allwise_sample.to_table()

ra,dec
deg,deg
float64,float64
202.5200113,47.2171496
202.4712600,47.2299357
202.4640701,47.2767995
202.4455931,47.2585766
202.5054408,47.2285282
202.4334361,47.2192841
202.4379914,47.2615984
202.5442330,47.2059442


In [88]:
async_srv = pyvo.dal.TAPService('https://irsa.ipac.caltech.edu/TAP')
data_type_name = async_srv.run_async("""
            SELECT DATA_TYPE 
            FROM INFORMATION_SCHEMA.COLUMNS
            WHERE 
                TABLE_NAME   = 'fp_psd'   
                """)

DALServiceError: Cannot wait for job completion. Job is not active!